In [2]:
import os
import glob
import pandas as pd
import numpy as np 
import pingouin as pg 
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from  itertools import combinations

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

warnings.filterwarnings("ignore")

In [3]:
curr_dir = os.getcwd()
avg_dir = curr_dir +"/AVG" 

rating_data = pd.read_csv( avg_dir + "/Dynamic_Rating.csv", index_col=0).values
rating_data.shape

(52, 16)

In [4]:
data_list = os.listdir(avg_dir)
data_list = sorted([ data for data in data_list if not data.startswith(".") and data != 'Dynamic_Rating.csv' and not data.endswith(".npy")])

avg_data = { str(x).split(sep=".")[0] : pd.read_csv(f"{avg_dir}/{x}", index_col=0).values for x in data_list}

avg_arr = [ ]

for ind, x in avg_data.items() : 
    #avg_arr.append(x.mean(axis=0))
    avg_arr.append(x.mean(axis=1)) # 52명 데이터를 평균내는 경우
    #avg_arr.append(x.flatten())

avg_arr = np.array(avg_arr).T

for key in avg_data.keys() : 
    print (key)
    print(avg_data[key].shape)
    print("-"*20)

print(avg_arr.shape)

Blinking
(52, 16)
--------------------
Fixation_Duration
(52, 16)
--------------------
Fixation_Num
(52, 16)
--------------------
Saccade_Duration
(52, 16)
--------------------
left_iris
(52, 16)
--------------------
left_pupil
(52, 16)
--------------------
right_iris
(52, 16)
--------------------
right_pupil
(52, 16)
--------------------
(52, 8)


In [5]:
# Dynamic ratign DF

#arr = np.concatenate([avg_arr, rating_data.mean(axis=0).reshape(-1,1)], axis=1)
arr = np.concatenate([avg_arr, rating_data.mean(axis=1).reshape(-1,1)], axis=1) # 52명 데이터를 평균내는 경우
#arr = np.concatenate([avg_arr, rating_data.flatten().reshape(-1,1)], axis=1)

print( "arr shape : ", arr.shape)

names = list(avg_data.keys())
names.append('DynamicRating')

#df = pd.DataFrame(arr, columns= names )
df = pd.DataFrame(np.concatenate([avg_arr, rating_data.mean(axis=1).reshape(-1,1)], axis=1 ), columns= names )# 52명 데이터를 평균내는 경우
#df = pd.DataFrame(arr, columns=names)

print("Dataframe shape :",df.shape)


arr shape :  (52, 9)
Dataframe shape : (52, 9)


In [6]:
def min_max_scaler( series) :
    min = series.min()
    max = series.max()
    x = (series - min ) / ( max - min ) 
    return x 

# df = df.apply( min_max_scaler)

In [7]:
combination = [ ]

for num in range(1,9) :
    
    for com in combinations(avg_data.keys(), num):
        
        x = '' 
        for factor in com : 
            x += "+"
            x += factor
        x = x[1:]

        m = ols(f"DynamicRating ~ {x}", data=df).fit()

        #if m.f_pvalue < 0.05 :

        combination.append([x, str(np.sqrt(m.rsquared).round(3)), str(m.rsquared.round(3)), str(m.f_pvalue.round(3))])

combination_df = pd.DataFrame(np.array(combination), columns= ["Combination", "R", "R2", "ANOVA_p-val"])
combination_df

,Combination,R,R2,ANOVA_p-val
0,Blinking,0.314,0.099,0.023
1,Fixation_Duration,0.087,0.008,0.539
2,Fixation_Num,0.122,0.015,0.388
3,Saccade_Duration,0.165,0.027,0.241
4,left_iris,0.018,0.0,0.901
...,...,...,...,...
250,Blinking+Fixation_Duration+Fixation_Num+left_i...,0.377,0.142,0.416
251,Blinking+Fixation_Duration+Saccade_Duration+le...,0.403,0.163,0.312
252,Blinking+Fixation_Num+Saccade_Duration+left_ir...,0.392,0.154,0.355
253,Fixation_Duration+Fixation_Num+Saccade_Duratio...,0.385,0.148,0.385


In [8]:
combination_df[combination_df['R2'] == combination_df['R2'].max()]

,Combination,R,R2,ANOVA_p-val
249,Blinking+Fixation_Duration+Fixation_Num+Saccad...,0.492,0.242,0.076
254,Blinking+Fixation_Duration+Fixation_Num+Saccad...,0.492,0.242,0.122
